In [1]:
from typing import List, Optional, Union, Literal, TypeVar
from pydantic import BaseModel, ValidationError, \
            validator, root_validator,Field, PrivateAttr, HttpUrl
from uuid import UUID

In [ ]:
# https://resource-watch.github.io/doc-api/#what-is-the-resource-watch-api

ENVIRONMENT = Literal['production', 'staging', 'preproduction']
DATA_TYPE = Literal['raster','vector', 'deck']
APPLICATION = Literal['rw','gfw','prep', 'gfw-pro', 'aqueduct', 'ng',
                      'aqueduct-water-risk', 'forest-atlas', 'demo']
INCLUDES = Literal['layer', 'widget', 'metadata', 'vocabulary', 'user']
LAYERPROVIDERS = Literal['cartodb', 'gee','featureservice','wms', 'leaflet']
LAYERTDATAYPES = Literal['tabular', 'raster']
DATASETDATAPROVIDERS = Literal['csv','json','featureservice', 'wms', 'cartodb',
                               'gee', 'vector']
DATASETCONNECTORTYPES = Literal['rest','document']

_excludeList = ['createdAt', 'updatedAt','clonedHost', 'errorMessage', 
                'taskId', 'status', 'sources','userId', 'slug', 'dataset', 
                'layer', 'widget', 'metadata', 'vocabulary']

In [2]:
class LayerConfigV4(BaseModel):
    type: DATA_TYPE
    lmMetadata: Optional[Union[dict, None]] = {"version": "4.0"}
    source: Optional[Union[dict, None]]
    render: Optional[Union[dict, None]]
    deck: Optional[Union[List[dict], None]]
    
    class Config:
        extra = 'allow'
        underscore_attrs_are_private = True

class LegendItem(BaseModel):
    name: str
    value: Optional[Union[str, None]]
    color: Optional[Union[str, None]]
    icon: Optional[Union[str, None]]
    
    class Config:
        extra = 'allow'
        underscore_attrs_are_private = True

class LegendConfig(BaseModel):
    type: Literal['basic', 'gradient', 'choropleth']
    items: List[LegendItem]
    unit: Optional[str] = None
    
    class Config:
        extra = 'allow'
        underscore_attrs_are_private = True

class InteractionConfig(BaseModel):
    type: Literal['gridjson', 'intersection', 'geojson']
    config: dict
    output: List[dict]
    
    class Config:
        extra = 'allow'
        underscore_attrs_are_private = True
        
class ApplicationConfig(BaseModel):
    name:str = 'John Doe'
    
    class Config:
        extra = 'allow'
        underscore_attrs_are_private = True

class LayerAttributes(BaseModel):
    '''
    
    Attributes
    ----------
    name : str
        Name of the layer
    description : str
        Description of the layer
    application : str
        Application that the layer belongs to
    dataset : str
        Dataset that the layer belongs to
    layerProvider : str
        Layer provider that the layer belongs to

    '''
    name: Optional[Union[str, None]] = Field(...)
    _slug: Optional[Union[str, None]] = PrivateAttr()
    _userId: Optional[Union[str, None]] = PrivateAttr()
    _datasetId: Optional[UUID] = PrivateAttr()
    description: Optional[Union[str, None]]
    application: List[APPLICATION] = ['rw']
    iso: Optional[list] = []
    provider: Optional[Union[str,LAYERPROVIDERS, None]] = ''
    type: Optional[Union[str,LAYERTDATAYPES, None]]
    default: bool = False
    protected: bool = False
    published: bool = False
    env: ENVIRONMENT = 'staging'
    #@TODO: Add specific field type for layerConfig, legendConfig ....
    layerConfig: Optional[Union[LayerConfigV4, dict]] = {}
    legendConfig: Optional[dict] = {}
    interactionConfig: Optional[dict] = {}
    applicationConfig: Optional[dict] = {}
    staticImageConfig: Optional[dict] = {}
    createdAt: Optional[str]
    updatedAt: Optional[str]
    # @TODO: extend for metadata and vocabulary
    # metadata: Optional[List[Metadata]]
    # vocabulary: Optional[List[Vocabulary]]
    
    def __init__(self, **data):
        super().__init__(**data)
        # this could also be done with default_factory
        self._slug = data.get('slug')
        self._userId = data.get('userId')
        self._datasetId = data.get('dataset')
    
    class Config:
        extra = 'ignore'
        underscore_attrs_are_private = True

class Layer(BaseModel):
    '''
    This class makes reference to a [RW API layer asset](
    https://resource-watch.github.io/doc-api/reference.html#layer-reference)
    ...

    Attributes
    ----------
    id: str
        The layer id
    type: str
        The layer type
    attributes: LayerAttributes object
        The layer attributes

    Methods
    -------
    '''
    id: Optional[UUID]
    type: Literal['layer'] = 'layer'
    attributes: LayerAttributes
    _url: Optional[HttpUrl]
        
    class Config:
        extra = 'ignore'
        underscore_attrs_are_private = True
        validate_assignment = True
        smart_union = True
    
    @root_validator(skip_on_failure=True)
    def _url(cls, values):
        id_val = values.get("id")
        dataset_id = values.get("attributes")._datasetId
        if id_val and dataset_id:
            values['_url'] = f'https://api.resourcewatch.org/v1/dataset/{dataset_id}/layer/{id_val}'
        else:
            values['_url'] = None
        return values
    
    def _repr_markdown_(self):
        if self._url:
            return f'''**{self.type.capitalize()}**: [{self.attributes.name}]({self._url})'''
        else:
            return f'''**{self.type.capitalize()}**: {self.attributes.name}'''

In [3]:
class WidgetAttributes(BaseModel):
    '''
    This class makes reference to a [RW API layer asset](
    https://resource-watch.github.io/doc-api/reference.html#layer-reference)
    '''
    name: Optional[Union[str, None]] = Field(...)
    _slug: Optional[Union[str, None]] = PrivateAttr()
    _userId: Optional[Union[str, None]] = PrivateAttr()
    _datasetId: Optional[UUID] = PrivateAttr()
    description: Optional[Union[str, None]]
    application: List[APPLICATION] = ['rw']
    iso: Optional[list]
    default: bool = False
    verified: bool = False
    protected: bool = False
    template: bool = False
    published: bool = False
    freeze: bool = False
    defaultEditableWidget: bool = False
    env: Optional[ENVIRONMENT]
    thumbnailUrl: Optional[Union[str, None]]
    queryUrl: Optional[Union[str, None]]
    widgetConfig: Optional[dict] = {} # in RW app this are either a vega spec with some widget editor flavours or widget editor spec
    createdAt: Optional[str]
    updatedAt: Optional[str]
    # @TODO: extend for metadata and vocabulary
    # metadata: Optional[List[Metadata]]
    # vocabulary: Optional[List[Vocabulary]]
    
    def __init__(self, **data):
        super().__init__(**data)
        # this could also be done with default_factory
        self._slug = data.get('slug')
        self._userId = data.get('userId')
        self._datasetId = data.get('dataset')
    
    class Config:
        extra = 'ignore'
        underscore_attrs_are_private = True
    

class Widget(BaseModel):
    '''
    This class makes reference to a [RW API layer asset](
    https://resource-watch.github.io/doc-api/reference.html#layer-reference)
    '''
    id: Optional[UUID]
    type: Literal['widget']
    attributes: WidgetAttributes
        
    class Config:
        extra = 'ignore'
        underscore_attrs_are_private = True
        validate_assignment = True
    
    @root_validator(skip_on_failure=True)
    def _url(cls, values):
        id_val = values.get("id")
        dataset_id = values.get("attributes")._datasetId
        if id_val:
            values['_url'] = f'https://api.resourcewatch.org/v1/dataset/{dataset_id}/widget/{id_val}'
        return values
    
    def _repr_markdown_(self):
        return f'''**{self.type.capitalize()}**: [{self.attributes.name}]({self._url})'''

In [4]:
class DatasetAttributes(BaseModel):    
    name: Optional[Union[str, None]] = Field(...)
    _slug: Optional[Union[str, None]] = PrivateAttr()
    _userId: Optional[Union[str, None]] = PrivateAttr()
    type: Optional[Union[str,Literal['tabular', 'raster'], None]]
    subtitle: Optional[Union[str, None]]
    application: List[APPLICATION] = ['rw']
    dataPath: Optional[Union[str, None]]
    attributesPath: Optional[Union[str, None]]
    connectorType: DATASETCONNECTORTYPES = 'rest'
    provider: DATASETDATAPROVIDERS = 'cartodb'
    connectorUrl: Optional[Union[str, None]]
    sources: Optional[list]
    tableName: Optional[Union[str, None]]
    status: Optional[Union[str, None]]
    published: bool = False
    overwrite: bool = False
    mainDateField: Optional[Union[str, None]]
    env: Optional[ENVIRONMENT]
    applicationConfig: Optional[dict]
    geoInfo: Optional[bool] = True
    protected: Optional[bool] = False
    legend: Optional[dict]
    clonedHost: Optional[dict]
    errorMessage: Optional[str]
    taskId: Optional[Union[str, None]]
    createdAt: Optional[str]
    updatedAt: Optional[str]
    dataLastUpdated: Optional[Union[str, None]]
    widgetRelevantProps: Optional[list]
    layerRelevantProps: Optional[list]
    layer: Optional[List[Layer]]
    # TODO: extend for widgets, metadata and vocabulary
    widget: Optional[List[Widget]]
    # metadata: Optional[List[Metadata]]
    # vocabulary: Optional[List[Vocabulary]]
    
    def __init__(self, **data):
        super().__init__(**data)
        # this could also be done with default_factory
        self._slug = data.get('slug')
        self._userId = data.get('userId')
    
    class Config:
        extra = 'ignore'
        underscore_attrs_are_private = True
    
    @root_validator
    def connectorType_provider_validator(cls, values):
        test = {'rest':['featureservice', 'wms', 'cartodb', 'gee', 'vector'],
                'document':['csv','json'],
                'wms':['wms']}
        connectorType, provider = values.get('connectorType'), values.get('provider')
        if provider not in test[connectorType]:
            raise ValueError(f'provider value: {provider} for connectorType value: {connectorType} is not right')

        return values
    
    @root_validator
    def provider_gee_validator(cls, values):
        provider, tableName = values.get('provider'), values.get('tableName')
        if provider == 'gee' and not tableName:
            raise ValueError(f'provider gee requires tableName')

        return values
    
class Dataset(BaseModel):
    '''
    This class makes reference to a [RW API dataset asset](
    https://resource-watch.github.io/doc-api/reference.html#dataset-reference)
    '''
    id: Optional[UUID]
    _url: Optional[HttpUrl]
    type: Optional[Union[Literal['dataset'], None]] = 'dataset'
    attributes: Optional[Union[DatasetAttributes, dict]]   
    
    def __init__(self, **data):
        super().__init__(**data)
    
    class Config:
        extra = 'ignore'
        underscore_attrs_are_private = True
        validate_assignment = True
    
    @root_validator(skip_on_failure=True)
    def _url(cls, values):
        id_val = values.get("id")
        if id_val:
            values['_url'] = f'https://api.resourcewatch.org/v1/dataset/{values.get("id")}'
        return values
    
    def _repr_markdown_(self):
        return f'''**{self.type.capitalize()}**: [{self.attributes.name}]({self._url})'''

In [5]:
APIRESOURCE = TypeVar('APIRESOURCE', Dataset, Layer, Widget)